In [2]:
from pathlib import Path
import tensorflow as tf
import sys

sys.path.append('/kaggle/input/axiom-utils')
import llm_components as lc

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

## Paths

In [4]:
directory = Path('/kaggle/input')
data_dir = directory / 'wikitext-103' / 'wikitext-103'
train_path = data_dir / 'wiki.train.tokens'
valid_path = data_dir / 'wiki.valid.tokens'
test_path = data_dir / 'wiki.test.tokens'
tokenizer_path = directory / 'axiom-utils' / 'sp_tokenizer.model'
# the reason for 2 checkpoint directories is because 
# kaggle reads from input directory, but saves in working directory
checkpoint_restore_dir = directory / 'axiom-utils' / 'checkpoints'
checkpoint_save_dir = Path('/kaggle/working/checkpoints')

## Hyper Parameters

In [5]:
SEQUENCE_LEN = 256    # Context size
SHIFT = 16
BATCH_SIZE = 64
N_EMBEDS = 512
N_HEADS = 8
N_BLOCKS = 8
STEPS_PER_EPOCH = 3000
VAL_STEPS = 200

## Loading Data

In [6]:
sp = lc.load_sp_tokenizer(str(tokenizer_path))
loader = lc.LMDatasetLoader(
    tokenizer= sp,
    shift= SHIFT,
    seq_len= SEQUENCE_LEN,
    batch_size= BATCH_SIZE,
    shuffle_buffer= 50_000
)

In [7]:
train_ds = loader.create(train_path, training= True)
valid_ds = loader.create(valid_path, training= False)
test_ds = loader.create(test_path, training= False)

I0000 00:00:1770741500.659343      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13757 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1770741500.665574      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13757 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [8]:
for item in train_ds.take(1):
    print(item)

(<tf.Tensor: shape=(64, 256), dtype=int32, numpy=
array([[  165,  1259,    37, ...,  3859,  7176,    37],
       [ 9113,   264,    36, ...,    67,  1273,   103],
       [ 4427,   124, 10922, ...,  3037,    53,  2873],
       ...,
       [11966,  7118,    92, ..., 15919,    11,  7671],
       [   37,  4855,  1917, ...,   783,    36,  4593],
       [    8,   265,  1421, ...,    11,  6958,  4723]], dtype=int32)>, <tf.Tensor: shape=(64, 256), dtype=int32, numpy=
array([[ 1259,    37,   143, ...,  7176,    37,   108],
       [  264,    36,    31, ...,  1273,   103, 14600],
       [  124, 10922,  2238, ...,    53,  2873,   247],
       ...,
       [ 7118,    92,     6, ...,    11,  7671,  1872],
       [ 4855,  1917,    11, ...,    36,  4593,  2010],
       [  265,  1421,    29, ...,  6958,  4723,   148]], dtype=int32)>)


## Strategy, callbacks, & some required calculations

In [9]:
vocab_size = sp.get_piece_size()
vocab_size

16000

In [10]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [11]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir= 'logs',
    histogram_freq= 1,
    update_freq= 100,    # every 100 batch
    embeddings_freq= 1
)

In [12]:
target_tokens = 500_000_000
steps_per_epoch = 3000
token_per_step = BATCH_SIZE * SEQUENCE_LEN
token_per_chunk = steps_per_epoch * token_per_step
total_steps = target_tokens // token_per_step
warmup_steps = int(total_steps * 0.05)  # 5%
print(f'Total target tokens: {target_tokens:3,}')
print(f'Steps per epoch: {steps_per_epoch:3,}')
print(f'Token per step: {token_per_step:3,}')
print(f'Token per chunk/epoch: {token_per_chunk:3,}')
print(f'Total steps for cosine decay: {total_steps:3,}')
print(f'Warmup steps for cosine decay: {warmup_steps:3,}')

Total target tokens: 500,000,000
Steps per epoch: 3,000
Token per step: 16,384
Token per chunk/epoch: 49,152,000
Total steps for cosine decay: 30,517
Warmup steps for cosine decay: 1,525


## Transformer Model

In [13]:
with strategy.scope():
    # creating model
    model = lc.GPT(
        vocab_size= vocab_size,
        seq_len= SEQUENCE_LEN,
        n_embeds= N_EMBEDS,
        n_heads= N_HEADS,
        n_blocks= N_BLOCKS
    )

    # lr schedule
    lr_schedule = lc.WarmupCosine(
        base_lr= 3e-4,
        warmup_steps= warmup_steps,
        total_steps= total_steps
    )

    # optimizer
    optimizer = tf.keras.optimizers.AdamW(
        learning_rate= lr_schedule,
        weight_decay= 0.01,
        beta_2= 0.95,
        clipnorm= 1.0
    )
    
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits= True     # softmax is handled by loss function
    )

    # compiling
    model.compile(
        optimizer= optimizer,
        loss= loss_fn,
        metrics= [lc.Perplexity()]
    )

    # passing dummy input to build model
    dummy = tf.zeros((1, SEQUENCE_LEN), dtype= tf.int32)
    _ = model(dummy, training= False)

    # checkpoint logic
    checkpoint = tf.train.Checkpoint(
        model= model,
        optimizer= optimizer
    )

    latest_ch = tf.train.latest_checkpoint(checkpoint_restore_dir)
    if latest_ch:
        print('Restoring State from', latest_ch)
        checkpoint.restore(latest_ch).expect_partial()
        print(
            'Step:', optimizer.iterations.numpy(),
            'LR:', lr_schedule(optimizer.iterations).numpy()
        )

    else:
        print('No Checkpoint found, random initialization.')

    manager = tf.train.CheckpointManager(
        checkpoint,
        checkpoint_save_dir,
        max_to_keep= 1
    )

Restoring State from /kaggle/input/axiom-utils/checkpoints/ckpt-2
Step: 6000 LR: 0.0002844365


In [14]:
model.summary()

Model: "gpt"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (1, 256, 512)          │     8,192,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (256, 512)             │       131,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block               │ ?                      │     3,150,848 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_1             │ ?                      │     3,150,848 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_2             │ ?                      │     3,150,848 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_3             │ ?                      │     3,150,848 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_4             │ ?                      │     3,150,848 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_5             │ ?                      │     3,150,848 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_6             │ ?                      │     3,150,848 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_7             │ ?                      │     3,150,848 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_16          │ ?                      │         1,024 │
│ (LayerNormalization)            │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,530,880 (127.91 MB)

 Trainable params: 33,530,880 (127.91 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
history = model.fit(
    train_ds, 
    steps_per_epoch= steps_per_epoch,
    epochs= 1,
    validation_data= valid_ds,
    validation_steps= VAL_STEPS,
    callbacks= [tensorboard_cb]
)

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [16]:
test_loss, test_perplexity = model.evaluate(test_ds)
print(f'{test_loss = }\n{test_perplexity = }')

282/282 ━━━━━━━━━━━━━━━━━━━━ 102s 360ms/step - loss: 5.5952 - perplexity: 301.0281
test_loss = 5.699486255645752
test_perplexity = 338.4040832519531


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


In [17]:
manager.save()

'/kaggle/working/checkpoints/ckpt-3'

In [18]:
import subprocess

subprocess.run(['zip', '-r', 'working_dir.zip', '/kaggle/working'], stdout= subprocess.DEVNULL)

CompletedProcess(args=['zip', '-r', 'working_dir.zip', '/kaggle/working'], returncode=0)